In [0]:
import tensorflow as tf
import numpy as np
import keras as K

In [22]:
# structure: 
'''
1. model
x -> conv1 -> fc -> prediction || cross-entropy

conv1 -> lat1 -> rec || rec_loss
conv1 -> lat1 -> 1.char || 1char_loss
1char_loss -> LSTM -> 


'''


'\n1. model\nx -> conv1 -> fc -> prediction || cross-entropy\n\nconv1 -> lat1 -> rec || rec_loss\nconv1 -> lat1 -> 1.char || 1char_loss\n1char_loss -> LSTM -> \n\n\n'

In [0]:
class model_RGB:
    def __init__(self, num_labels, z_channels, image_size=(32, 32), encoder_depth=5, dense_hidden_sizes=(50,100,150), dense_acts=(tf.nn.sigmoid, tf.nn.sigmoid, tf.nn.sigmoid)):
        self.num_labels = num_labels
        self.image_size = image_size
        self.dense_hidden_sizes = dense_hidden_sizes
        self.encoder_depth = encoder_depth
        self.dense_acts = dense_acts
        
        self.z_shape = (*[int(a/pow(2, encoder_depth)) for a in image_size] ,z_channels)
        print("expected z_shape", self.z_shape)
        
        
    def encoder(self, x, k_size, pool_size, reuse=False):
        with tf.variable_scope('ENCODER'):
            if reuse:
                tf.get_variable_scope().reuse_variables()
        
            nn = [None for _ in range(self.encoder_depth)]
            #c = [*reversed([[*[int(a*pow(2,d)) for a in self.z_shape[:-1]], int(self.z_shape[2]/pow(2, d))] for d in range(self.encoder_depth+1)])]
            c = [*reversed([int(self.z_shape[2]/pow(2, d)) for d in range(self.encoder_depth+1)])]
            print('encoder channels: ', c)
                    
              
            nn[0] = tf.layers.conv3d(x, c[1], k_size, padding='SAME', kernel_initializer=tf.initializers.random_normal, bias_initializer=tf.initializers.random_normal, activation=tf.nn.relu)
            nn[0] = tf.layers.max_pooling3d(nn[0], pool_size, strides=pool_size, padding='SAME')
            for d in range(1, self.encoder_depth-1):
                nn[d] = tf.layers.conv3d(nn[d-1], c[d+1], k_size, padding='SAME', kernel_initializer=tf.initializers.random_normal, bias_initializer=tf.initializers.random_normal, activation=tf.nn.relu)
                nn[d] = tf.layers.max_pooling3d(nn[d], pool_size, strides=pool_size, padding='SAME')
            nn[-1] = tf.layers.conv3d(nn[-2], c[-1], k_size, padding='SAME', kernel_initializer=tf.initializers.random_normal, bias_initializer=tf.initializers.random_normal, activation=tf.nn.relu)
            nn[-1] = tf.layers.max_pooling3d(nn[-1], pool_size, strides=pool_size, padding='SAME')
        
        print("conv shape: ", nn[-1].shape)
        return nn[-1]
        
        
    def dense(self, x, reuse=False):
        with tf.variable_scope('DENSE'):
            if reuse:
                tf.get_variable_scope().reuse_variables()

            h_layers = [None for _ in range(len(self.dense_hidden_sizes))]

            h_layers[0] = tf.layers.dense(x, self.dense_hidden_sizes[0], activation=self.dense_acts[0])

            for i in range(len(self.dense_hidden_sizes)-1):
                h_layers[i+1] = tf.layers.dense(h_layers[i], self.dense_hidden_sizes[i+1], activation=self.dense_acts[i+1])
                h_layers[i+1] = tf.layers.dropout(h_layers[i+1], rate=0.13)
                
            out = tf.layers.dense(h_layers[-1], self.num_labels, activation=tf.nn.softmax)

        print('expected output size: ', out.shape)
        return out
        
    

In [0]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
ITERATIONS = 1000
IMAGE_SHAPE = (28*28, 1)
BATCH_SIZE = 1
IN_CHANNELS = 1
ENCODER_DEPTH = 6
NUM_LABELS = 10
Z_CHANNELS = 64
LR = 0.004
WD = 0.29

In [107]:
tf.reset_default_graph()

M = model_RGB(NUM_LABELS, Z_CHANNELS, encoder_depth=ENCODER_DEPTH, dense_hidden_sizes=(IMAGE_SHAPE[0], 350, 500, 250, 150, 100, 50, 20), dense_acts=[tf.nn.relu for _ in range(8)])

X = tf.placeholder(name='X', dtype=tf.float32, shape=[BATCH_SIZE, *IMAGE_SHAPE, IN_CHANNELS])
label = tf.placeholder(name='label', dtype=tf.float32, shape=[BATCH_SIZE, NUM_LABELS])

#z = M.encoder(X, k_size=[6, 6, 3], pool_size=[3, 3, 1])

pred = M.dense(tf.transpose(X))

expected z_shape (0, 0, 64)
expected output size:  (1, 1, 784, 10)


In [108]:
# use K.utils.to_categorical(labels, NUM_LABELS) to convert to onehot
# np.transpose(data.reshape, axes=[0,2,1,3]) for rotation
train_variables = tf.trainable_variables()

loss = tf.losses.softmax_cross_entropy(label, pred)

opt = tf.contrib.opt.AdamWOptimizer(learning_rate=LR, weight_decay=WD).minimize(loss, var_list=train_variables)
#opt = tf.train.GradientDescentOptimizer(learning_rate=LR).minimize(loss, var_list=train_variables)

ValueError: ignored

In [104]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
  
    for i in range(ITERATIONS):

        rnd_ind = np.random.randint(0, x_train.shape[0])
        
#        img = x_train[rnd_ind][np.newaxis, ..., np.newaxis, np.newaxis]
        img = x_train[rnd_ind].reshape(IMAGE_SHAPE)[np.newaxis, ..., np.newaxis]
        labl = K.utils.to_categorical(y_train[rnd_ind], NUM_LABELS)[np.newaxis, ...]
        
        currentLoss = 0
        
        
        _, L = sess.run([opt, loss], feed_dict={"X:0": img, "label:0": labl})
        
  
        print(L)
        
        
#         for j, _ in enumerate(x_train):
#           x = x_train[j][np.newaxis, ..., np.newaxis, np.newaxis]
#           y = K.utils.to_categorical(y_train[j], NUM_LABELS)[np.newaxis, ...]
          
#           _, currentLoss = sess.run([opt, loss], feed_dict={"X:0": x, "label:0": y})
          
#           if j % 100 == 0:
#              print("itteration: \t {} loss: \t {}".format(i, currentLoss))
        

1806.9941
1805.048
1805.4648
1804.5076
1805.4373
1804.8204
1805.0236
1805.5559
1805.6277
1805.3944
1804.7458
1804.8054
1805.0325
1804.7423
1805.0599
1805.7505
1804.925
1805.7871
1805.8135
1805.0239
1805.377
1805.8907
1805.0414
1805.0199
1805.2242
1805.3053
1805.1395
1805.3684
1805.3925
1805.1992
1805.0952
1805.1914
1805.2455
1805.0326
1805.1012
1805.3318
1805.1163
1805.1162
1805.213
1805.0336
1805.5223
1805.4183
1805.3795
1805.1158
1805.2694
1805.0079
1805.1675
1804.9635
1805.2609
1804.8918
1805.2626
1805.1642
1805.0956
1805.1771
1804.8645
1805.4028
1804.8406
1805.1968
1804.9641
1805.2452
1805.296
1805.4183
1805.667
1805.5343
1805.1754
1805.1998
1805.5616
1805.2842
1805.1406
1805.3677
1805.2096
1805.2115
1805.1498
1805.2393
1805.4362
1805.3799
1805.296
1805.2968
1805.203
1805.224
1805.0448
1805.355
1805.4426
1804.9369
1805.2694
1805.1874
1805.2688
1805.2568
1805.1139
1805.2992
1805.307
1805.2123
1805.2029
1805.4369
1805.3668
1805.1017
1805.2216
1805.5115
1805.1396
1805.0685
1805.4249
1

KeyboardInterrupt: ignored